<a href="https://colab.research.google.com/github/wlin988/Colab/blob/master/5_1_Pretrained_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/
!ls -lta

/content/drive
total 8
drwx------ 5 root root 4096 Oct 28 19:57 'My Drive'
drwx------ 2 root root 4096 Oct 28 19:57  .Trash


In [0]:
#!cp -r "My Drive"  my_drive

In [0]:
#!git clone https://github.com/joosthub/PyTorchNLPBook.git

In [5]:
pip install annoy

     |████████████████████████████████| 645kB 2.7MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310400 sha256=1912ca5809b8ec62edd1c3591e37c1035f24b44244a5c97d9f341682a1a56c7d
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy


In [6]:
%cd /content/drive/My Drive/
!ls -lta

/content/drive/My Drive
total 4053
drwx------ 2 root root    4096 Oct 27 20:33  folder
drwx------ 6 root root    4096 Oct 27 20:28  PyTorchNLPBook
drwx------ 2 root root    4096 Oct 26 17:41 'Colab Notebooks'
-rw------- 1 root root     151 Oct 25 21:04 'Untitled document (3).gdoc'
-rw------- 1 root root     151 Oct 25 09:30 'Untitled document (2).gdoc'
-rw------- 1 root root 1453568 Sep 16  2014 '2A-4 -  6WINDGate Multi-Core Networking SW - Level 2 v5.4.ppt'
-rw------- 1 root root   56383 Jun 27  2014 '5th Spelling W & Def 91-100 (3).pdf'
-rw------- 1 root root     151 Apr 30  2014 'Untitled document (1).gdoc'
-rw------- 1 root root     151 Apr 30  2014 'Untitled document.gdoc'
-rw------- 1 root root  269382 Mar 17  2014 'Print from a Chromebook with Google Cloud Print - YouTube.pdf'
-rw------- 1 root root   69344 Mar 17  2014  Google.pdf
-rw------- 1 root root   56471 Mar 17  2014 'Google (1).pdf'
-rw------- 1 root root   56471 Mar 17  2014 'Google (2).pdf'
-rw------- 1 root root   69

In [7]:
%cd data
!ls -lta

[Errno 2] No such file or directory: 'data'
/content/drive/My Drive
total 4053
drwx------ 2 root root    4096 Oct 27 20:33  folder
drwx------ 6 root root    4096 Oct 27 20:28  PyTorchNLPBook
drwx------ 2 root root    4096 Oct 26 17:41 'Colab Notebooks'
-rw------- 1 root root     151 Oct 25 21:04 'Untitled document (3).gdoc'
-rw------- 1 root root     151 Oct 25 09:30 'Untitled document (2).gdoc'
-rw------- 1 root root 1453568 Sep 16  2014 '2A-4 -  6WINDGate Multi-Core Networking SW - Level 2 v5.4.ppt'
-rw------- 1 root root   56383 Jun 27  2014 '5th Spelling W & Def 91-100 (3).pdf'
-rw------- 1 root root     151 Apr 30  2014 'Untitled document (1).gdoc'
-rw------- 1 root root     151 Apr 30  2014 'Untitled document.gdoc'
-rw------- 1 root root  269382 Mar 17  2014 'Print from a Chromebook with Google Cloud Print - YouTube.pdf'
-rw------- 1 root root   69344 Mar 17  2014  Google.pdf
-rw------- 1 root root   56471 Mar 17  2014 'Google (1).pdf'
-rw------- 1 root root   56471 Mar 17  2014 

In [8]:

%cd /content/drive/"My Drive"/PyTorchNLPBook/chapters/chapter_5/data
!pwd
!ls -lta

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data
/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data
total 52
drwx------ 2 root root  4096 Oct 28 18:59 .ipynb_checkpoints
-rw------- 1 root root  1961 Oct 28 17:58 get-all-data.sh
-rw------- 1 root root  1572 Oct 27 20:28 download.py
-rw------- 1 root root  6148 Oct 27 20:28 .DS_Store
-rw------- 1 root root   572 Oct 27 20:28 README.md
-rw------- 1 root root 12288 Oct 27 20:28 .README.md.swp
drwx------ 2 root root  4096 Oct 27 20:28 ag_news
drwx------ 2 root root  4096 Oct 27 20:28 books
drwx------ 2 root root  4096 Oct 27 20:28 glove
drwx------ 2 root root  4096 Oct 27 20:28 nmt
drwx------ 2 root root  4096 Oct 27 20:28 surnames
drwx------ 2 root root  4096 Oct 27 20:28 yelp


In [9]:
!chmod 777 get-all-data.sh
!ls -lta

total 52
drwx------ 2 root root  4096 Oct 28 18:59 .ipynb_checkpoints
-rwx------ 1 root root  1961 Oct 28 17:58 get-all-data.sh
-rw------- 1 root root  1572 Oct 27 20:28 download.py
-rw------- 1 root root  6148 Oct 27 20:28 .DS_Store
-rw------- 1 root root   572 Oct 27 20:28 README.md
-rw------- 1 root root 12288 Oct 27 20:28 .README.md.swp
drwx------ 2 root root  4096 Oct 27 20:28 ag_news
drwx------ 2 root root  4096 Oct 27 20:28 books
drwx------ 2 root root  4096 Oct 27 20:28 glove
drwx------ 2 root root  4096 Oct 27 20:28 nmt
drwx------ 2 root root  4096 Oct 27 20:28 surnames
drwx------ 2 root root  4096 Oct 27 20:28 yelp


In [0]:
! ./get-all-data.sh

In [11]:
!mkdir -p glove
!pwd
!ls

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data
ag_news  download.py	  glove  README.md  yelp
books	 get-all-data.sh  nmt	 surnames


In [12]:
%cd /content/drive/"My Drive"/PyTorchNLPBook/chapters/chapter_5/data/glove
!ls

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data/glove
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt


In [13]:
!wget -nc http://nlp.stanford.edu/data/glove.6B.zip

File ‘glove.6B.zip’ already there; not retrieving.



In [14]:
!ls
!unzip -n glove.6B.zip

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt
Archive:  glove.6B.zip


In [15]:
!pwd
%cd ../..
!ls

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data/glove
/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5
5_1_Pretrained_Embeddings.ipynb  5_3_doc_classification  model_storage
5_2_CBOW			 data			 README.md


In [16]:
%run 5_1_Pretrained_Embeddings.ipynb

Building Index!
Finished!
man : he :: woman : she
man : he :: woman : never
fly : plane :: sail : ship
fly : plane :: sail : vessel
cat : kitten :: dog : puppy
cat : kitten :: dog : toddler
cat : kitten :: dog : sleds
blue : color :: dog : cat
blue : color :: dog : animal
blue : color :: dog : breed
leg : legs :: hand : fingers
leg : legs :: hand : ears
leg : legs :: hand : stick
toe : foot :: finger : hand
toe : foot :: finger : attached
toe : foot :: finger : apart
talk : communicate :: read : instructions
talk : communicate :: read : communicating
talk : communicate :: read : transmit
blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator
man : king :: woman : queen
man : king :: woman : monarch
man : king :: woman : throne
man : doctor :: woman : nurse
man : doctor :: woman : physician
fast : fastest :: small : smallest
fast : fastest :: small : largest
fast : fastest :: small : among


In [0]:
#import os
#from getpass import getpass
#import urllib

#cmd = 'git clone https://github.com/joosthub/PyTorchNLPBook.git'
#os.system(cmd)

In [0]:
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

In [0]:
class PreTrainedEmbeddings(object):
    """ A wrapper around pre-trained word vectors and their use """
    def __init__(self, word_to_index, word_vectors):
        """
        Args:
            word_to_index (dict): mapping from word to integers
            word_vectors (list of numpy arrays)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("Building Index!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("Finished!")
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """Instantiate from pre-trained vector file.
        
        Vector file should be of the format:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        Args:
            embedding_file (str): location of the file
        Returns: 
            instance of PretrainedEmbeddigns
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
                
        return cls(word_to_index, word_vectors)
    
    def get_embedding(self, word):
        """
        Args:
            word (str)
        Returns
            an embedding (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]

    def get_closest_to_vector(self, vector, n=1):
        """Given a vector, return its n nearest neighbors
        
        Args:
            vector (np.ndarray): should match the size of the vectors 
                in the Annoy index
            n (int): the number of neighbors to return
        Returns:
            [str, str, ...]: words that are nearest to the given vector. 
                The words are not ordered by distance 
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_and_print_analogy(self, word1, word2, word3):
        """Prints the solutions to analogies using word embeddings

        Analogies are word1 is to word2 as word3 is to __
        This method will print: word1 : word2 :: word3 : word4
        
        Args:
            word1 (str)
            word2 (str)
            word3 (str)
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)

        # now compute the fourth word's embedding!
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship

        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = set([word1, word2, word3])
        closest_words = [word for word in closest_words 
                             if word not in existing_words] 

        if len(closest_words) == 0:
            print("Could not find nearest neighbors for the computed vector!")
            return
        
        for word4 in closest_words:
            print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

In [20]:
embeddings = PreTrainedEmbeddings.from_embeddings_file('data/glove/glove.6B.100d.txt')

Building Index!
Finished!


In [21]:
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : never


In [22]:
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship
fly : plane :: sail : vessel


In [23]:
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

cat : kitten :: dog : puppy
cat : kitten :: dog : toddler
cat : kitten :: dog : sleds


In [24]:
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

blue : color :: dog : cat
blue : color :: dog : animal
blue : color :: dog : breed


In [25]:
embeddings.compute_and_print_analogy('leg', 'legs', 'hand')

leg : legs :: hand : fingers
leg : legs :: hand : ears
leg : legs :: hand : stick


In [26]:
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

toe : foot :: finger : hand
toe : foot :: finger : attached
toe : foot :: finger : apart


In [27]:
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

talk : communicate :: read : instructions
talk : communicate :: read : communicating
talk : communicate :: read : transmit


In [28]:
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator


In [29]:
embeddings.compute_and_print_analogy('man', 'king', 'woman')

man : king :: woman : queen
man : king :: woman : monarch
man : king :: woman : throne


In [30]:
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician


In [31]:
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')

fast : fastest :: small : smallest
fast : fastest :: small : largest
fast : fastest :: small : among


In [32]:
%cd 5_2_CBOW/data
!ls
!chmod 777 get-all-data.sh
! ./get-all-data.sh

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/5_2_CBOW/data
ag_news  download.py	  glove  README.md  yelp
books	 get-all-data.sh  nmt	 surnames


In [33]:
!mkdir -p glove
%cd glove
!wget -nc http://nlp.stanford.edu/data/glove.6B.zip
!unzip -n glove.6B.zip


/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/5_2_CBOW/data/glove
File ‘glove.6B.zip’ already there; not retrieving.

Archive:  glove.6B.zip


In [34]:
%cd ../../..

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5


In [35]:
%run 5_2_CBOW/5_2_munging_frankenstein.ipynb

LookupError: ignored